In [1]:
import pandapower as pp #import pandapower
import numpy as np
import pandas as pd
import pandapower.networks as pn 
from pandapower.estimation import estimate
import pandapower.plotting as plot
%matplotlib inline
import matplotlib.pyplot as plt

## IEEE 39-Bus System
The “IEEE 39 bus system is well known as 10-machine New-England Power System. Generator 1 represents the aggregation of a large number of generators.
<img src="IEEE39.png" width="600">

Calls the pickle file case39.p which data origin is PYPOWER. This network was published the first time in
G. Bills et al., On-line stability analysis study, RP 90-1, E. P. R. I. North American Rockwell Corporation,
Edison Electric Institute, Ed. IEEE Press, Oct. 1970,. Some more information about this network are
given by Illinois University case 39. Because the Pypower data origin proposes vn_kv=345 for all nodes
the transformers connect node of the same voltage level.

## Load the pandapower built-in case 39 and run the powerflow computation

In [2]:
net = pn.case39()
pp.runpp(net)
net

This pandapower network includes the following parameter tables:
   - bus (39 elements)
   - load (21 elements)
   - gen (9 elements)
   - ext_grid (1 element)
   - line (35 elements)
   - trafo (11 elements)
   - polynomial_cost (10 elements)
   - bus_geodata (39 elements)
 and the following results tables:
   - res_ext_grid (1 element)
   - res_load (21 elements)
   - res_gen (9 elements)
   - res_bus (39 elements)
   - res_line (35 elements)
   - res_trafo (11 elements)

## Load the bus data from the Excel sheet

In [3]:
df_bus = pd.read_excel('39busbar-data-second-try.xlsx', sheet_name='BusEntry_normal')
df_bus['busnnumber'] = df_bus['busnnumber']-1
df_bus_real = df_bus[['busnnumber', 'busFinalVoltage', 'busFinalDegree', 'busLoadMW', 'busLoadMVAR', 'busGenerationMW', 'busGenerationMVAR']]

## Bus data

In [4]:
df_bus_real

,busnnumber,busFinalVoltage,busFinalDegree,busLoadMW,busLoadMVAR,busGenerationMW,busGenerationMVAR
0,0,1.047356,-8.438685,0.000000,0.000000,0.000000,0.000000
1,1,1.048736,-5.753762,0.000000,0.000000,0.000000,0.000000
2,2,1.030173,-8.598549,322.000000,2.400000,0.000000,0.000000
3,3,1.003863,-9.606667,500.000000,184.000000,0.000000,0.000000
4,4,1.005311,-8.611863,0.000000,0.000000,0.000000,0.000000
5,5,1.007672,-7.949683,0.000000,0.000000,0.000000,0.000000
6,6,0.997001,-10.123823,233.800003,84.000000,0.000000,0.000000
7,7,0.996020,-10.615381,522.000000,176.000000,0.000000,0.000000
8,8,1.028226,-10.321987,0.000000,0.000000,0.000000,0.000000
9,9,1.017151,-5.427126,0.000000,0.000000,0.000000,0.000000


## Add noise and measurments

In [5]:
bus_real_arr = df_bus_real.values
sigma_vm = 0.01
sigma_p = 0.0001
sigma_q = 0.0001
measurment_index = bus_real_arr[:,0].astype(int)
measurment_vm = bus_real_arr[:,1] + np.random.normal(0,sigma_vm,39)
measurment_p = (bus_real_arr[:,5]- bus_real_arr[:,3])*1000+ np.random.normal(0,sigma_p,39)
measurment_q = (bus_real_arr[:,6]- bus_real_arr[:,4])*1000 + np.random.normal(0,sigma_q,39)

In [6]:
for i in range(39):
    pp.create_measurement(net, "v", "bus", measurment_vm[i], sigma_vm, measurment_index[i])
    pp.create_measurement(net, "p", "bus", measurment_p[i], sigma_p, measurment_index[i])        
    pp.create_measurement(net, "q", "bus", measurment_q[i], sigma_q, measurment_index[i]) 
net.measurement

,name,type,element_type,value,std_dev,bus,element
0,None,v,bus,1.052957,0.0100,0,None
1,None,p,bus,0.000183,0.0001,0,None
2,None,q,bus,0.000041,0.0001,0,None
3,None,v,bus,1.043550,0.0100,1,None
4,None,p,bus,0.000008,0.0001,1,None
5,None,q,bus,0.000116,0.0001,1,None
6,None,v,bus,1.035083,0.0100,2,None
7,None,p,bus,-322000.000050,0.0001,2,None
8,None,q,bus,-2400.000191,0.0001,2,None
9,None,v,bus,1.003934,0.0100,3,None


## State estimation

In [7]:
success = estimate(net, init='flat', tolerance=1e-06, maximum_iterations=100,ref_power=100000000)
print(success)

True


In [8]:
m_index = np.array([0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,  2, 20, 21, 
                   22, 23, 24, 25, 26, 27, 28, 29,  3, 30, 31, 32, 33, 34, 35, 
                   36, 37, 38,  4, 5, 6,  7,  8,  9], dtype=np.int)
index_dict={k:m_index[k] for k in range(39)}

In [9]:
net.res_bus_est.rename(index=index_dict)

,vm_pu,va_degree,p_kw,q_kvar
0,1.046856,-8.448762,-0.916314,0.025664
1,1.048360,-5.762009,-0.911767,0.023271
10,1.012087,-6.291679,-0.908630,0.020228
11,0.999531,-6.251036,7499.091417,88000.019523
12,1.013706,-6.104983,-0.908510,0.019933
13,1.011145,-7.665657,-0.911555,0.019201
14,1.014809,-7.745807,319999.089025,153000.017501
15,1.031216,-6.195692,328999.093537,32300.017686
16,1.033062,-7.310961,-0.908069,0.018716
17,1.030444,-8.234280,157999.088664,30000.019466


## Next Steps: compute confidence interval of estimation and plot them by error bar